## NaPDI machine reading knowledge graph instance closure

Closure run for SemRep predicates. If only REACH predications involved, use other notebook to generate triples/RDF graph instead of running this notebook

In [1]:
## Set up the CLIPS environment
import clips
env = clips.Environment()

MIN_PREDICATION_BELIEF = 0
MIN_TRANSITIVE_BELIEF = 0  # chosen because it retains most depth 1 transitive inferences over semmed  

## NOTE: BE SURE TO CLEAR test-inference.ntriples EACH TIME BEFORE RUNNING!!
## This accomplishes that
f = open("test-inference.ntriples",'w')
f.close()

### Rules

In [2]:
## Example
env.clear()
env.reset()

env.eval('(open "test-inference.ntriples" writeFile "a")')

env.build("""
(deftemplate oav
 (slot object)
 (slot attribute)
 (slot value)
 (slot predNS)
 (slot inferred (default No))
 (slot belief (default 0.0)))
""")

## Transitive rule 
env.build("""
(defrule transitive
  "a simple transitivity rule"
  (oav (object ?o)
       (attribute ?pred&:(member$ ?pred (create$ predisposes causes influence precedes part_of)))
       (value ?s)
       (predNS RO)
       (inferred No)
       (belief ?b1))
  (oav (object ?s)
       (attribute ?pred)
       (value ?q)
       (predNS RO)
       (inferred No)
       (belief ?b2))
   (test (>= (* ?b1 ?b2) {}))
  =>
  (assert (oav (object ?o)
               (attribute ?pred)
               (value ?q)
               (inferred Yes)
               (predNS RO)
               (belief (* ?b1 ?b2))))
  
  (printout writeFile (format nil "<http://dikb.org/ad#%s><http://purl.obolibrary.org/obo/%s><http://dikb.org/ad#%s>.%n" ?o ?pred ?q))   
)
""".format(MIN_TRANSITIVE_BELIEF))
# NOTE: add this line to RHS to see the belief scores:
# (printout writeFile (format nil "b1: %f, b2: %f, belief: %f>.%n" ?b1 ?b2 (* ?b1 ?b2))) 

## simplerule for symmetric relationships 
env.build("""
(defrule symmetric
  "a simple symmetry rule"
  (oav (object ?o)
       (attribute ?pred&:(member$ ?pred (create$ interacts_with association associated_with coexists_with compared_with same_as)))
       (value ?s)
       (predNS RO)
       (inferred No)
       (belief ?b))
  =>
  (assert (oav (object ?s)
               (attribute ?pred)
               (value ?o)
               (inferred Yes)
               (predNS RO)
               (belief ?b)))
  
  (printout writeFile (format nil "<http://dikb.org/ad#%s><http://purl.obolibrary.org/obo/%s><http://dikb.org/ad#%s>.%n" ?o ?pred ?s))  
)
""")


### Facts 

In [3]:
# Load the TSV data files for eidos and reach
stmtL = [] 

## TODO: Handle MachineReadingOutputs/alz_eidos_predicates_NO_MAP.tsv seperately
##       It has no CUI mappings

fl = [
      'MachineReadingOutputs/SEMREP_AD_13365_modified.tsv',
      'MachineReadingOutputs/alz_13365_eidos_predicates_modified.tsv',
      'MachineReadingOutputs/alz_13365_reach_predicates_modified.tsv'
      ]

for fnm in fl:
    f = open(fnm,'r')
    buf = f.read()
    f.close()
    
    tL = buf.split('\n')    
    stmtL = stmtL + tL[1:] # skip header lines here
    print("INFO: loaded file:{} with {} records.".format(fnm,len(tL)))

print("INFO: total number of records loaded: {}".format(len(stmtL)))

INFO: loaded file:MachineReadingOutputs/SEMREP_AD_13365_modified.tsv with 80898 records.
INFO: loaded file:MachineReadingOutputs/alz_13365_eidos_predicates_modified.tsv with 9069 records.
INFO: loaded file:MachineReadingOutputs/alz_13365_reach_predicates_modified.tsv with 4970 records.
INFO: total number of records loaded: 94934


In [4]:
originalTriplesF = open('original-triples.ntriples','w')

resourceD = {}
resourceDinv = {}
rcnt = 0
fctStrD = {}
semTypeD = {}
labelsD = {}

[INDEX,PMID,SUBJECT_SOURCE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_TYPE,SUBJECT_SCORE,PREDICATE,OBJECT_SOURCE,OBJECT_CUI,OBJECT_NAME,OBJECT_TYPE,OBJECT_SCORE,BELIEF,SENTENCE,SOURCE] = range(0,16)

for stmt in stmtL: 
    stmtSplt = stmt.split('\t')
    #check this
    if stmt == "" or len(stmtSplt) != 16:
        continue
        
    if float(stmtSplt[BELIEF]) < MIN_PREDICATION_BELIEF:
        continue
    
    (subj, pred, obj) = (stmtSplt[SUBJECT_CUI],
                        stmtSplt[PREDICATE].lower().strip(),
                        stmtSplt[OBJECT_CUI])
        
    # Subj and obj need to be validly formatted CUIs
    if subj == None or obj == None or subj == "" or obj == "" or subj[0] != 'C' or obj[0] != 'C':
        continue

    # only write out and/or do inference over some predicates
    if pred not in ['affects','associated_with','augments',
                    'causes','coexists_with','complicates',
                    'disrupts','inhibits','interacts_with',
                    'part_of','precedes', 'predisposes','prevents',
                    'treats',
                    'association', 
                    'influence', 
                    'modification',
                    'regulateactivity',
                    'regulateamount',
                    'acetylation',
                    'deacetylation',
                    'defarnesylation',
                    'degeranylgeranylation',
                    'deglycosylation',
                    'dehydroxylation',
                    'demethylation'
                    'demyristoylation',
                    'depalmitoylation',
                    'dephosphorylation',
                    'deribosylation',
                    'desumoylation',
                    'deubiquitination',
                    'farnesylation',
                    'geranylgeranylation',
                    'glycosylation',
                    'hydroxylation',
                    'methylation',
                    'myristoylation',
                    'palmitoylation',
                    'phosphorylation',
                    'ribosylation',
                    'sumoylation',
                    'ubiquitination',
                    'activation','inhibition','increaseamount','decreaseamount'                    
                   ]:        
        continue
        
    # write the original triple to file, regardless of the predicate
    originalTriplesF.write("<http://dikb.org/ad#{}><http://purl.obolibrary.org/obo/{}><http://dikb.org/ad#{}>.\n".format(subj,pred,obj))
        
    # Track the subject and objet names
    subjName = stmtSplt[SUBJECT_NAME]
    objName = stmtSplt[OBJECT_NAME]
    if not labelsD.get(subjName):
        labelsD[subj] = subjName
    
    if not labelsD.get(objName):
        labelsD[obj] = objName
    
    # Track the semantic types
    semTypesStr = stmtSplt[SUBJECT_TYPE]
    if semTypesStr.find('[') == -1:
        if not semTypeD.get(subj):
            semTypeD[subj] = [semTypesStr.strip()]
        else:
            semTypeD[subj] = semTypeD[subj].append(semTypesStr.strip())
    else:
        semTypesStr = semTypesStr.replace("'",'').replace('[','').replace(']','')
        semTypesL = [x.strip() for x in semTypesStr.split(',')]
        if not semTypeD.get(subj):
            semTypeD[subj] = semTypesL
        else:
            semTypeD[subj] = semTypeD[subj] + semTypesL
            
    semTypesStr = stmtSplt[OBJECT_TYPE]
    if semTypesStr.find('[') == -1:
        if not semTypeD.get(obj):
            semTypeD[obj] = [semTypesStr.strip()]
        else:
            semTypeD[obj] = semTypeD[obj].append(semTypesStr.strip())
    else:
        semTypesStr = semTypesStr.replace("'",'').replace('[','').replace(']','')
        semTypesL = [x.strip() for x in semTypesStr.split(',')]
        if not semTypeD.get(obj):
            semTypeD[obj] = semTypesL
        else:
            semTypeD[obj] = semTypeD[obj] + semTypesL
        
    if not resourceD.get(subj):
        resourceD[subj] = 'r{}'.format(rcnt)
        resourceDinv['r{}'.format(rcnt)] = subj
        rcnt += 1
    
    if not resourceD.get(obj):
        resourceD[obj] = 'r{}'.format(rcnt)
        resourceDinv['r{}'.format(rcnt)] = obj
        rcnt += 1    
    
        
    fctStr = """
(oav (object {})
     (attribute {})
     (value {})
     (predNS {})
     (belief {})
)""".format(resourceD[subj], pred, resourceD[obj], 'RO', float(stmtSplt[BELIEF]))
    
    if not fctStrD.get(fctStr): 
        env.assert_string(fctStr)
        fctStrD[fctStr] = 1

# write the human readable labels as triples
for (e,st) in labelsD.items():
    if st:
        originalTriplesF.write('<http://dikb.org/ad#{}><http://www.w3.org/2000/01/rdf-schema#label> "{}".\n'.format(e,st.replace('"','')))

# Write the semantic types as triples
for (e,st) in semTypeD.items():
    if st:
        stSet = set(st)
        for elt in stSet:
            originalTriplesF.write("<http://dikb.org/ad#{}><http://www.w3.org/1999/02/22-rdf-syntax-ns#type><http://umls.org/st/#{}>.\n".format(e,elt.replace('"','')))
    
originalTriplesF.close()

In [5]:
len(fctStrD.keys())

22507

In [6]:
i = 0
for fact in env.facts():
    print(fact)
    if i == 20:
        break
    i += 1

(initial-fact)
(oav (object r0) (attribute coexists_with) (value r1) (predNS RO) (inferred No) (belief 0.8))
(oav (object r1) (attribute predisposes) (value r2) (predNS RO) (inferred No) (belief 0.8))
(oav (object r3) (attribute precedes) (value r4) (predNS RO) (inferred No) (belief 0.8))
(oav (object r5) (attribute associated_with) (value r6) (predNS RO) (inferred No) (belief 0.8))
(oav (object r7) (attribute causes) (value r5) (predNS RO) (inferred No) (belief 0.8))
(oav (object r8) (attribute predisposes) (value r9) (predNS RO) (inferred No) (belief 0.8))
(oav (object r10) (attribute causes) (value r11) (predNS RO) (inferred No) (belief 0.8))
(oav (object r12) (attribute coexists_with) (value r13) (predNS RO) (inferred No) (belief 0.8))
(oav (object r13) (attribute treats) (value r14) (predNS RO) (inferred No) (belief 0.8))
f-10    (oav (object r12) (attribute treats) (value r14) (predNS RO) (inferred No) (belief 0.8))
f-11    (oav (object r15) (attribute affects) (value r16) (predN

In [7]:
env.run()  
# count of inferences by cutoff for transitive belief score: 0.3 and 0.5 = 155214; 0.6 = 155214 (most of semmed is still in); 
#               0.7 = 143746 (loses most of semmed); 0.8 = 123742; 0.9 = 33244; 1.0 = 6439 (all transitive inferences dropped)

89409

The output tells use how many RHS made changes to working memory

In [8]:
env.eval("close(writeFile)")

'close'

In [9]:
# predicate to RO mapping 
predMapD = {
    'regulateactivity':'',
    'regulateamount':'',
    'phosphorylation':'RO_0002447',
    'dephosphorylation':'GO_0006470',
    'ubiquitination':'RO_0002480',
    'deubiquitination':'',
    'sumoylation':'',
    'desumoylation':'',
    'hydroxylation':'GO_0018126',
    'dehydroxylation':'',
    'acetylation':'GO_0006473',
    'deacetylation':'GO_0006476',
    'glycosylation':'GO_0006486',
    'deglycosylation':'GO_0006517',
    'farnesylation':'',
    'defarnesylation':'',
    'geranylgeranylation':'',
    'degeranylgeranylation':'',
    'palmitoylation':'',
    'depalmitoylation':'',
    'myristoylation':'',
    'demyristoylation':'',
    'ribosylation':'',
    'deribosylation':'',
    'methylation':'GO_0006479',
    'demethylation':'GO_0006482',
    'activation':'RO_0002436',
    'inhibition':'RO_0002449',
    'increaseamount':'RO_0002429',
    'decreaseamount':'RO_0002212'
}

import re

f = open('test-inference.ntriples','r')
buf = f.read()
f.close()
rsL = buf.split('\n')

#rgx.findall(s)
#[('http://purl.org/vocab/relationship/', 'influence')]

rgx = re.compile('(http://purl.obolibrary.org/obo/)([a-z_]+)')
for i in range(0,len(rsL)):
    if rsL[i] == "":
        continue
        
    ml = rgx.findall(rsL[i])
    if len(ml) != 1:
        print('ERROR: could not match on predicate regex: {}'.format(rsL[i]))
        continue
        
    (uri,predicate) = ml[0]
    rsL[i] = rsL[i].replace(predicate, predMapD[predicate])
    

f = open('inferred-transitive-and-symmetric.ntriples','w')
rgx = re.compile('#(r[0-9]+)')
for it in rsL:
    keyL = rgx.findall(it)
    newTr = it
    for k in keyL:
        if resourceDinv.get(k):
            newTr = newTr.replace('http://dikb.org/ad#' + k, 'http://dikb.org/ad#' + resourceDinv[k])            
        else:
            print('ERROR: key not found in resourceDinv: {}'.format(k))
    f.write(newTr + '\n')
f.close()


In [10]:
# Do the same RO mapping for the original predications
f = open('original-triples.ntriples','r')
buf = f.read()
f.close()
rsL = buf.split('\n')

rgx = re.compile('(http://purl.obolibrary.org/obo/)([a-z_]+)')
f = open('original-triples-RO-mapped.ntriples','w')
for i in range(0,len(rsL)):
    if rsL[i] == "":
        continue
        
    if rsL[i].find('<http://www.w3.org/2000/01/rdf-schema#label>') != -1 or rsL[i].find('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>') != -1:
        f.write(rsL[i] + '\n')    
        continue
        
    ml = rgx.findall(rsL[i])
    if len(ml) != 1:
        print('ERROR: could not match on predicate regex: {}'.format(rsL[i]))
        continue
        
    (uri,predicate) = ml[0]
    f.write(rsL[i].replace(predicate, predMapD[predicate]) + '\n')    
f.close()

In [16]:
## create a file with triples to map the UMLS CUIs to the target ontologies used in the Pheknowlator KG ()
all_cui_to_codeD = {}
umls_mapped_cnt = 0
reach_mapped_cnt = 0
bioportal_mapped_cnt = 0

# start by loading all GO and HPO UMLS CUIs
# ex: C2247561|GO|GO:0034023|"5-(carboxyamino)imidazole ribonucleotide mutase activity"
f = open('cui_to_ontology_maps/go_hpo_maps.csv','r')
buf = f.read()
f.close
lns = buf.split('\n')
for ln in lns[1:]: # skip header
    if ln == "":
        continue
        
    (cui, ont, code, string) = ln.replace('""','').split('|')
    if not all_cui_to_codeD.get(cui):
        all_cui_to_codeD[cui] = ['http://purl.obolibrary.org/obo/' + code.replace(':','_')]
    else:
        all_cui_to_codeD[cui].append('http://purl.obolibrary.org/obo/' + code.replace(':','_'))

umls_mapped_cnt = len(all_cui_to_codeD.keys())
        
# add mappings to OBO from REACH 
# ex: quercetin	"('CHEBI', 'CHEBI:16243')"	"{'CHEBI': 'CHEBI:16243', 'TEXT': 'Quercetin', 'HMDB': 'HMDB0005794', 'MESH': 'D011794', 'PUBCHEM': '5280343', 'CAS': '117-39-5', 'DRUGBANK': 'DB04216', 'CHEMBL': 'CHEMBL50'}"	C0034392
f = open('cui_to_ontology_maps/reach_bio_mapping.tsv','r')
buf = f.read()
f.close
lns = buf.split('\n')
for ln in lns[1:]: # skip header
    if ln == "":
        continue
        
    (agent_name,grounding,other_mapping,cui) = ln.replace('"','').split('\t')
    
    (ont_str,code) = eval(grounding)
    if ont_str == None or (code != None and code.find(':') == -1):
        continue
    
    if not all_cui_to_codeD.get(cui):
        all_cui_to_codeD[cui] = ['http://purl.obolibrary.org/obo/' + code.replace(':','_')]
    else:
        all_cui_to_codeD[cui].append('http://purl.obolibrary.org/obo/' + code.replace(':','_'))

reach_mapped_cnt = len(all_cui_to_codeD.keys()) - umls_mapped_cnt
        
## now, add mappings that came from a query of the bioportal annotator using the 
## string labels of the UMLS CUIs from the MRCONSO table (see cui_to_ontology_maps/non_go_hpo_strings_annotator_query.curl)

# Bioportal annotator response
import json
f = open('cui_to_ontology_maps/non_go_hpo_strings_annotator_response.json','r')
map_d_l = json.load(f)
f.close()

# all none GO or HPO UMLS CUIs mentioned in the triples and the string labels
f = open('cui_to_ontology_maps/non_go_hpo_strings.csv','r')
buf = f.read()
f.close
lns = buf.split('\n')
strToCuiD = {}
cuiToStrD = {}
for ln in lns[1:]: # skip header
    if ln == "":
        continue
        
    (cui, s) = ln.replace('""','').split('|')
    strToCuiD[s.upper()] = cui
    cuiToStrD[cui] = s.upper()

for d in map_d_l:
    mapped_to = d['annotatedClass']['@id']
    for mapped_from_d in d['annotations']:
            if not strToCuiD.get(mapped_from_d['text'].upper()):
                continue
            else:
                cui = strToCuiD[mapped_from_d['text'].upper()]
                if not all_cui_to_codeD.get(cui):
                    all_cui_to_codeD[cui] = [mapped_to]
                else:
                    if mapped_to not in all_cui_to_codeD[cui]:
                        all_cui_to_codeD[cui].append(mapped_to)

bioportal_mapped_cnt = len(all_cui_to_codeD.keys()) - reach_mapped_cnt
                        
print('CUI mapping total: {} ({} from umls, {} from reach-to-obo, {} from bioportal)'.format(len(all_cui_to_codeD.values()),
                                                                                            umls_mapped_cnt,
                                                                                            reach_mapped_cnt,
                                                                                            bioportal_mapped_cnt))
ontCounts = {}
for cui,m in all_cui_to_codeD.items():
    for elt in m:
        ontId = elt.split('/')[-1].split('_')[0]
        if ontCounts.get(ontId):
            ontCounts[ontId] += 1
        else:
            ontCounts[ontId] = 1

for ontId,count in ontCounts.items():
        print('{}:{}'.format(ontId,count))
        

f = open('cui_mapping_triples.ntriples','w')
for cui,m in all_cui_to_codeD.items():
    for elt in m:
        f.write("<http://dikb.org/ad#{}><http://dikb.org/ad#obo_mapping><{}>.\n".format(cui,elt))
f.close()        

CUI mapping total: 2504 (844 from umls, 394 from reach-to-obo, 2110 from bioportal)
GO:685
UBERON:133
HP:472
CHEBI:1068
PW:5
CL:20
DOID:101
CLO:7
EFO:12
PR:639
PATO:16
VO:18
BFO:5
NCBITaxon:44
OBI:2
RO:1
IAO:3
cellline#human:1
SO:1
NCIT:1
PRO:1
